In [ ]:
# default_exp nlp.fasttext

# FastText - UMAP - HDBSCAN

> API details.

In [ ]:
import os
import time
import sys
import inspect

import pandas as pd
import numpy as np

import datetime

try:
    from fasttext import train_unsupervised
    import fasttext
except:
    from fastText import train_unsupervised
    import fastText
    
import umap.umap_ as umap
import hdbscan

import plotly.graph_objs as go
import plotly.offline as py
import plotly.graph_objs as go
from IPython.display import Image,display

In [ ]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Get Data

In [ ]:
fasttext_data_folder = "../data/fasttext_data"
train_data = os.path.join(fasttext_data_folder,"text_file.text")

In [ ]:
text_df = pd.read_csv(train_data, names=["itemDesc"])
text_df.head()

,itemDesc
0,MINI DONUTS FOURRES ASSORTI X6
1,NAVETTE ASSORTIMENT X30
2,BRIOCHE MOUNA 400G
3,BRIOCHETTE ST GENIX X4 LOCAL
4,BRIOCHE ST GENIX LOCAL


## FastText training

In [ ]:
cbow_model = os.path.join(fasttext_data_folder,"model_cbow.bin")
skipgram_model = os.path.join(fasttext_data_folder,"model_skipgram.bin")


**train_unsupervised parameters**
    * input             # training file path (required)
    lr                # learning rate [0.1]
    dim               # size of word vectors [100]
    ws                # size of the context window [5]
    epoch             # number of epochs [5]
    minCount          # minimal number of word occurences [1]
    minCountLabel     # minimal number of label occurences [1]
    minn              # min length of char ngram [0]
    maxn              # max length of char ngram [0]
    neg               # number of negatives sampled [5]
    wordNgrams        # max length of word ngram [1]
    loss              # loss function {ns, hs, softmax, ova} [softmax]
    bucket            # number of buckets [2000000]
    thread            # number of threads [number of cpus]
    lrUpdateRate      # change the rate of updates for the learning rate [100]
    t                 # sampling threshold [0.0001]
    label             # label prefix ['__label__']
    verbose           # verbose [2]
    pretrainedVectors # pretrained word vectors (.vec file) for supervised learning []

In [ ]:
model_to_use = "skipgram"

if model_to_use == "skipgram":
    # Skipgram model :
    model_skipgram = train_unsupervised(train_data, model='skipgram')
    model_skipgram.save_model(skipgram_model)

else:
    #Cbow model :
    model_cbow = train_unsupervised(train_data, model='cbow')
    model_cbow.save_model(cbow_model)

In [ ]:
# Loading of the fastext pretained model
model_skipgram=fasttext.load_model(skipgram_model)

In [ ]:
model_skipgram.get_dimension()

100

In [ ]:
# The get_sentence_vector takes all vectors for all the words in the query, divide each of them by their respective norm, and then average all vectors together
def query_to_vector(col_query, model_fastText):
    vector = col_query.apply(lambda x:model_fastText.get_sentence_vector(x.replace('\n',' ')))
    return vector

In [ ]:
%timeit text_df['vector'] = query_to_vector(text_df['itemDesc'], model_skipgram)
del model_skipgram # We do not need the pretrained-vector in memory

21.1 ms ± 6.23 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
text_df.head(10)

,itemDesc,vector
0,MINI DONUTS FOURRES ASSORTI X6,"[0.02158853, 0.10800936, -0.014428361, -0.0226..."
1,NAVETTE ASSORTIMENT X30,"[0.01491683, 0.0463899, -0.042345867, -0.01604..."
2,BRIOCHE MOUNA 400G,"[-0.052592173, 0.0015068303, 0.0072030267, -0...."
3,BRIOCHETTE ST GENIX X4 LOCAL,"[-0.009621026, -0.016895643, -0.0072085024, -0..."
4,BRIOCHE ST GENIX LOCAL,"[-0.017043956, -0.03252602, 0.0002465509, -0.0..."
5,NAVETTE X20,"[0.026373366, -0.0044601816, -0.047210522, -0...."
6,TARTE AU SUCRE 6P LOCAL,"[-0.022768188, -0.087489955, -0.011846682, 0.0..."
7,MINI SUISSE X8,"[0.09550478, 0.01049311, 0.14422202, -0.133910..."
8,4 P RAIS+4 P CHOC.+4 CROIS. PB,"[0.011772488, 0.03474198, -0.043323327, 0.0313..."
9,X5 POCHE TALOAK,"[0.0113559365, 0.007889184, 0.0071355393, -0.0..."


## Dimension reduction with UMAP

In [ ]:
# We regroup all the vectors as a numpy array
vecs=text_df.vector.values
vecs=np.stack(vecs, axis=0)
vecs.shape

(583, 100)

In [ ]:
col_names = ["FT_"+str(x) for x in range(0,100)]
print(len(col_names))
test = pd.DataFrame(vecs, columns=col_names)

100


In [ ]:
pd.concat([text_df,test], axis=1)

,itemDesc,vector,FT_0,FT_1,FT_2,FT_3,FT_4,FT_5,FT_6,FT_7,FT_8,FT_9,FT_10,FT_11,FT_12,FT_13,FT_14,FT_15,FT_16,FT_17,FT_18,FT_19,FT_20,FT_21,FT_22,FT_23,FT_24,FT_25,FT_26,FT_27,FT_28,FT_29,FT_30,FT_31,FT_32,FT_33,FT_34,FT_35,FT_36,FT_37,FT_38,FT_39,FT_40,FT_41,FT_42,FT_43,FT_44,FT_45,FT_46,FT_47,FT_48,FT_49,FT_50,FT_51,FT_52,FT_53,FT_54,FT_55,FT_56,FT_57,FT_58,FT_59,FT_60,FT_61,FT_62,FT_63,FT_64,FT_65,FT_66,FT_67,FT_68,FT_69,FT_70,FT_71,FT_72,FT_73,FT_74,FT_75,FT_76,FT_77,FT_78,FT_79,FT_80,FT_81,FT_82,FT_83,FT_84,FT_85,FT_86,FT_87,FT_88,FT_89,FT_90,FT_91,FT_92,FT_93,FT_94,FT_95,FT_96,FT_97,FT_98,FT_99
0,MINI DONUTS FOURRES ASSORTI X6,"[0.02158853, 0.10800936, -0.014428361, -0.0226...",0.021589,0.108009,-0.014428,-0.022685,0.028280,0.087975,-0.041295,0.035083,-0.038354,0.059446,-0.016814,-0.067185,-0.009770,0.041939,0.018721,-0.032885,-0.008577,-0.075262,-0.076178,-0.031120,-0.031405,-0.009856,-0.019507,0.001981,-0.017272,0.034201,0.046960,0.013545,-0.010608,-0.015410,-0.001328,-0.051276,0.070788,-0.055409,0.026858,-0.114945,-0.011401,-0.041776,-0.007304,-0.002424,0.018402,-0.028861,-0.059076,-0.060262,0.035896,-0.014707,0.072236,0.066961,0.024299,0.021932,-0.020327,-0.039282,-0.077175,0.033308,0.003746,0.007083,0.005596,-0.037530,-0.006734,0.032848,0.109271,-0.067665,-0.014027,-0.016231,0.012769,-0.035605,-0.031053,-0.034607,-0.025912,-0.051481,0.035385,0.080805,-0.037025,0.019314,-0.025771,0.018008,0.075179,0.010719,-0.003863,0.003526,0.047260,-0.005215,0.038773,-0.001793,0.001866,-1.152315e-02,-0.029741,-0.032821,0.048355,0.006067,-0.015916,-0.074651,0.016909,-0.076681,0.034382,-0.009295,0.105053,-0.032185,-0.043723,-0.099093
1,NAVETTE ASSORTIMENT X30,"[0.01491683, 0.0463899, -0.042345867, -0.01604...",0.014917,0.046390,-0.042346,-0.016042,0.060447,-0.001797,-0.008045,0.154593,-0.047424,0.001132,0.032217,-0.059574,0.000717,0.001230,0.161351,-0.091212,-0.071427,0.090881,-0.095705,0.029092,-0.081243,0.044408,-0.016194,0.024331,-0.010957,-0.078223,0.084997,0.005291,-0.034417,-0.063888,-0.033734,0.063391,0.003938,-0.017342,-0.007127,-0.062274,-0.106297,-0.080983,-0.023470,-0.035458,0.023864,0.077621,0.040681,-0.087621,-0.048138,0.029328,-0.020172,-0.013593,-0.000041,-0.003258,0.059128,0.015102,-0.114664,-0.056938,-0.020718,-0.028130,-0.116602,0.071858,-0.002723,-0.021435,-0.057342,-0.058563,0.019870,-0.075311,-0.025875,0.038814,-0.069627,-0.000348,0.081428,-0.058699,0.071183,-0.005207,-0.045708,-0.052555,-0.041583,0.094484,0.106114,-0.012816,-0.102720,0.048042,-0.119289,0.007316,0.008341,0.129051,0.028428,-7.965221e-02,0.045516,-0.008542,-0.040991,-0.006551,-0.094721,-0.047693,0.011115,-0.078789,0.081529,0.060254,0.022350,-0.001472,-0.122700,-0.096947
2,BRIOCHE MOUNA 400G,"[-0.052592173, 0.0015068303, 0.0072030267, -0....",-0.052592,0.001507,0.007203,-0.075680,0.012186,0.064446,-0.018036,0.027879,0.003005,-0.021490,-0.051380,0.038776,-0.012552,-0.007170,0.048757,0.066186,0.004172,0.000296,0.013349,-0.029195,0.067025,0.082093,-0.071041,0.061429,-0.029163,-0.019110,-0.022450,-0.008152,0.015607,-0.009915,-0.005514,0.020968,-0.018824,-0.065464,-0.050438,0.037773,-0.017940,-0.011465,-0.148411,-0.020821,0.046366,0.064355,-0.054890,0.033843,-0.025245,0.047857,0.064039,0.028168,0.077153,-0.032028,-0.019151,-0.009380,-0.048796,-0.022810,-0.142978,-0.029365,0.026998,0.028427,0.018528,0.063950,-0.028166,-0.054642,-0.087238,-0.024618,-0.018250,-0.036454,0.023793,0.018883,-0.055154,0.056277,0.059927,0.017362,-0.042239,-0.019660,-0.082078,-0.076041,0.021497,0.087942,-0.014110,0.004494,-0.041924,0.027201,0.096872,0.105393,-0.008537,-6.275854e-02,-0.015903,0.058681,-0.071392,-0.024214,0.039547,-0.050930,-0.062387,-0.081011,-0.132281,-0.143439,-0.023694,-0.057964,0.012188,-0.110044
3,BRIOCHETTE ST GENIX X4 LOCAL,"[-0.009621026, -0.016895643, -0.0072085024, -0...",-0.009621,-0.016896,-0.007209,-0.020957,0.027357,0.004364,-0.031856,0.004262,-0.055088,-0.095801,-0.016329,0.009801,-0.006620,-0.030099,0.016967,-0.032480,0.027324,-0.014296,-0.017306,0.0

In [ ]:
fit = umap.UMAP(n_neighbors=30,min_dist=0.1,n_components=3,metric='cosine',random_state=42)
%time u = fit.fit_transform(vecs)

text_df['x']=u[:,0]
text_df['y']=u[:,1]
text_df['z']=u[:,2]

CPU times: user 5.83 s, sys: 107 ms, total: 5.94 s
Wall time: 6.27 s


In [ ]:
u.shape

(583, 3)

## 3D Vizualisation

In [ ]:
# This is the function to plot the queries in the embedding space.
# Here we reduce the embedding to a 3 dimensionnal space
def plot_cluster(df,iscolored=False,name='',interactive=True):
    
    if interactive:
        if iscolored:
            color=df['cluster'].values
        else: color = df['x']
        trace1 = go.Scatter3d(
            x=df['x'],
            y=df['y'],
            z=df['z'],

            mode='markers',
            marker=dict(
                size=3,
                color=color,                # set color to an array/list of desired values
                colorscale='Viridis',   # choose a colorscale
                opacity=0.3
            ),
            text=color
        )

        data = [trace1]
        layout = go.Layout(
            margin=dict(
                l=0,
                r=0,
                b=0,
                t=0
            )
        )
        fig = go.Figure(data=data, layout=layout)
        file='../data/fasttext_data/'+name+'.html'
        py.iplot(fig, filename=file)
        py.plot(fig, filename=file,auto_open=False)

In [ ]:
is_interactive=False
filename='umap_embedding_description'

if is_interactive:
    plot_cluster(text_df,False,filename)

## Clustering with HDBSCAN

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=5,min_samples=5)

In [ ]:
clusterer

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='euclidean', min_cluster_size=5, min_samples=5, p=None,
        prediction_data=False)

In [ ]:
%time text_df['cluster']= clusterer.fit_predict(u)

CPU times: user 28.7 ms, sys: 3.83 ms, total: 32.5 ms
Wall time: 30.9 ms


In [ ]:
text_df.groupby(['cluster'])['itemDesc'].apply(list).apply(len).sort_values(ascending=False)[0:20]

cluster
-1     161
 12     71
 8      33
 13     28
 18     27
 9      24
 20     22
 16     22
 11     19
 6      17
 10     15
 2      13
 14     12
 22     11
 23     11
 15     11
 5       9
 4       9
 3       9
 1       9
Name: itemDesc, dtype: int64

In [ ]:
# The cluster number -1 means actually that the algorithm considered it as noise.
# So we will remove the noise
denoised = text_df[text_df.cluster!=-1]

In [ ]:
denoised

,itemDesc,vector,x,y,z,cluster
0,MINI DONUTS FOURRES ASSORTI X6,"[0.02158853, 0.10800936, -0.014428361, -0.0226...",0.645584,-2.659455,-3.383080,9
2,BRIOCHE MOUNA 400G,"[-0.052592173, 0.0015068303, 0.0072030267, -0....",-3.299386,-4.585773,-2.454369,12
3,BRIOCHETTE ST GENIX X4 LOCAL,"[-0.009621026, -0.016895643, -0.0072085024, -0...",-2.899542,-4.460345,-4.154674,18
4,BRIOCHE ST GENIX LOCAL,"[-0.017043956, -0.03252602, 0.0002465509, -0.0...",-3.165269,-4.390998,-4.015838,18
7,MINI SUISSE X8,"[0.09550478, 0.01049311, 0.14422202, -0.133910...",-0.511436,-2.663689,-2.772304,0
12,BRIOCHE NANTERRE PB 200G,"[-0.057377636, -0.082840875, 0.024681043, -0.0...",-2.361974,-4.934412,-2.334155,12
14,MAXI BEIGNET POMME X2,"[-0.042263057, -0.040797967, -0.015237959, -0....",-0.207774,-3.361046,-4.676132,8
15,BOULE BRIOCHE CHOCOLAT X4,"[-0.009851376, 0.0010039834, -0.041003454, -0....",-1.273335,-5.335170,-2.631642,4
16,MINI VIENNOISERIE BIO X15,"[0.06694245, -0.072235204, -0.016080922, -0.05...",0.112263,-3.454064,-2.806441,9
17,MINI VIENNOISERIE X10,"[0.04730966, -0.057482123, 0.0504705, -0.09562...",-0.039886,-3.381804,-2.874100,9


In [ ]:
# Number of clusters
print('Total number of clusters: '+str(len(denoised.cluster.unique())))

Total number of clusters: 26


In [ ]:
clusters=denoised.groupby(['cluster'])['itemDesc'].apply(list)

In [ ]:
cluster_i = clusters[0]
print(len(cluster_i))
cluster_i

9


['MINI SUISSE X8',
 'MINI GOURMANDISES X8',
 '200G MINI BEIGNET ABRICOT X8',
 'MINI PANIER SALE X8',
 '200G MINI BEIGNET CARAMEL X8',
 '200G MINI BEIGNET CHOCOLAT X8',
 '200G MINI BEIGNET POMME X8',
 '200G MINI BEIGNET FRAMBOISE X8',
 '200G MINI BEIGNET CHOC BLAN X8']

In [ ]:
for i, cluster_i in enumerate(clusters):
    print("---------")
    print("cluster {} size:{}".format(i,len(cluster_i)))
    print(cluster_i)

---------
cluster 0 size:9
['MINI SUISSE X8', 'MINI GOURMANDISES X8', '200G MINI BEIGNET ABRICOT X8', 'MINI PANIER SALE X8', '200G MINI BEIGNET CARAMEL X8', '200G MINI BEIGNET CHOCOLAT X8', '200G MINI BEIGNET POMME X8', '200G MINI BEIGNET FRAMBOISE X8', '200G MINI BEIGNET CHOC BLAN X8']
---------
cluster 1 size:9
['500G GACHE PEPITES CHOCOLAT', 'SUISSE CHOCOLATX2', 'BRESSANE PEPITES CHOCO 12P LOC', 'BRESSANE PEPITES DE CHOCO LOC', 'CHINOIS PEPITES CHOCOLAT', 'TORSADE CHOCO X 2', 'AGNEAU PASCAL PEPITES', 'SUISSE PEPITES CHOCO X2', 'BRIOCHE PEPITES DE CHOCO. LOC']
---------
cluster 2 size:13
['POGNE DE ROMANS 500 CF', 'SAINT-GENIX 500 CF', '500G POGNE DE ROMANS', '500G MOUNA DELICES DAUPHINOIS', '500G GATEAU DE PAQUES AUX FRTS', 'MOUNA 500 CF', '500G POGNE DE ROMANS', 'CHINOIS CREME PATISSIERE 500G', '500G POGNE DE ROMANS', '500G POGNE DE ROMANS CUIT SURG', '3F.ABRICOT+3P.RAISIN+3P.CHOCO', '500G MOUNA CUIT SURGELE', 'AGNEAU DE PAQUES 170G']
---------
cluster 3 size:9
['220G OREILLETTE CR

## TF-IDF into the clusters

In [ ]:
def get_top_grams(cluster_j):
    tokens = nltk.word_tokenize(" ".join(cluster_j))

    #Create your bigrams
    tgs = nltk.trigrams(tokens)
    bgs = nltk.bigrams(tokens)

    #compute frequency distribution for all the bigrams in the text
    fdist = nltk.FreqDist(tgs)
    trigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
    fdist = nltk.FreqDist(bgs)
    bigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
    fdist = nltk.FreqDist(tokens)
    onegram = fdist.most_common(1)[0][0],fdist.most_common(1)[0][1]

    return trigram, bigram, onegram

In [ ]:
# Cluster description
for i, cluster_i in enumerate(clusters):
    print("---------")
    print("cluster {} size:{}".format(i,len(cluster_i)))
    print(get_top_grams(cluster_i))

---------
cluster 0 size:9


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(('MOUNA LOCAL GRIAUDE', 1), ('LOCAL GACHE', 2), ('LOCAL', 5))


In [ ]:
tokens = nltk.word_tokenize(" ".join(cluster_i))

#Create your bigrams
tgs = nltk.trigrams(tokens)
bgs = nltk.bigrams(tokens)

#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(tgs)
trigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
fdist = nltk.FreqDist(bgs)
bigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
fdist = nltk.FreqDist(tokens)
onegram = fdist.most_common(1)[0][0],fdist.most_common(1)[0][1]

print(trigram, bigram, onegram)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

('MOUNA LOCAL GRIAUDE', 1) ('LOCAL GACHE', 2) ('LOCAL', 5)


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_connectors.gcp.ipynb.
Converted 01_nlp.fasttext.ipynb.
Converted 02_forecasting.dataprep.ipynb.
Converted 03_models.catboost.ipynb.
Converted 04_nlp.nbsvm.ipynb.
Converted index.ipynb.
